In [1]:
import os
from osgeo import ogr

In [2]:
def create_proj_file(shapefile, spatialRef):
    """Create a projection file corresponding to the given shapefile."""
    path = os.path.splitext(shapefile)[0] + ".prj"
    with open(path, 'w') as proj_file:
        proj_file.write(spatialRef.ExportToWkt())
    return path


def create_ds_from_source(path, source_layer, source_layer_def, spatialRef):
    """Copy the source vector schema."""
    if os.path.exists(path):
        driver.DeleteDataSource(path)

    ds = driver.CreateDataSource(path)
    layer = ds.CreateLayer(source_layer.GetName(),  
                           geom_type=source_layer.GetGeomType())

    # Add input Layer Fields to the output Layer
    for i in range(source_layer_def.GetFieldCount()):
        field_def = source_layer_def.GetFieldDefn(i)
        layer.CreateField(field_def)
    # Get the Layer's Feature Definition
    layer_def = layer.GetLayerDefn()
    
    create_proj_file(path, spatialRef)
    
    # Return the Dataset, the Layer and the Layer Definition
    return ds, layer, layer_def


def split_source(train_path, test_path, source_layer, source_layer_def, spatialRef):
    """
    Given a shapefile path, split the poligons in two different shapefiles: train and test.
    
    Currently: just send half and half.
    
    """
    train_ds, train_layer, train_layer_def = create_ds_from_source(
        train_path, source_layer, source_layer_def, spatialRef
    )
    test_ds, test_layer, test_layer_def = create_ds_from_source(
        test_path, source_layer, source_layer_def, spatialRef
    )
    
    vectors = [
        (train_layer, train_layer_def),
        (test_layer, test_layer_def)
    ]
    current = 0

    # Add features to the ouput Layer
    for feat in source_layer:
        layer, layer_def = vectors[current]
        # Create output Feature
        new_feature = ogr.Feature(layer_def)

        # Add field values from input Layer
        for i in range(layer_def.GetFieldCount()):
            fieldDefn = layer_def.GetFieldDefn(i)
            new_feature.SetField(
                layer_def.GetFieldDefn(i).GetNameRef(), 
                feat.GetField(i)
            )

        geom = feat.GetGeometryRef()
        new_feature.SetGeometry(geom.Clone())
        # Add new feature to output Layer
        layer.CreateFeature(new_feature)

        current = (current+1)%2

    # Close DataSources

    train_ds.Destroy()
    test_ds.Destroy()

In [3]:
# Choose the files to split
vector_data_path = "data/bccba/VECTOR/Roi_Para_Clasificar/"
files = [f for f in os.listdir(vector_data_path) 
         if f.endswith(".shp") and not f.startswith('ROI')]
shapefiles = [os.path.join(vector_data_path, f) for f in files if f.endswith('.shp')]

In [4]:
driver = ogr.GetDriverByName('ESRI Shapefile')

for source_vector_path in shapefiles:
    source_vector_basename = os.path.basename(source_vector_path).split('.')[0]
    #data_date = source_vector_basename.split("_")[-1]
    
    source_ds = ogr.Open(source_vector_path)
    source_layer = source_ds.GetLayer()
    source_layer_def = source_layer.GetLayerDefn()

    spatialRef = source_layer.GetSpatialRef()
    spatialRef.MorphToESRI()

    train_data_dir = "/tmp"
    train_vector_basename = "%s_train" % source_vector_basename
    train_vector_path = os.path.join(train_data_dir, train_vector_basename + ".shp")

    test_data_dir = train_data_dir
    test_vector_basename = "%s_test" % source_vector_basename
    test_vector_path = os.path.join(test_data_dir, test_vector_basename + ".shp")
    
    split_source(
        train_vector_path, test_vector_path, source_layer, source_layer_def, spatialRef
    )
    print("Created from %s\n\t%s\n\t%s" % (source_vector_path, train_vector_path, test_vector_path))
    
    source_ds.Destroy()


Created from data/bccba/VECTOR/Roi_Para_Clasificar/ALFA_ROI_229_82_150201.shp
	/tmp/ALFA_ROI_229_82_150201_train.shp
	/tmp/ALFA_ROI_229_82_150201_test.shp
Created from data/bccba/VECTOR/Roi_Para_Clasificar/ALFA_ROI_229_82_150217.shp
	/tmp/ALFA_ROI_229_82_150217_train.shp
	/tmp/ALFA_ROI_229_82_150217_test.shp
Created from data/bccba/VECTOR/Roi_Para_Clasificar/ALFA_ROI_229_82_150321.shp
	/tmp/ALFA_ROI_229_82_150321_train.shp
	/tmp/ALFA_ROI_229_82_150321_test.shp
Created from data/bccba/VECTOR/Roi_Para_Clasificar/MAIZ_ROI_229_82_150217.shp
	/tmp/MAIZ_ROI_229_82_150217_train.shp
	/tmp/MAIZ_ROI_229_82_150217_test.shp
Created from data/bccba/VECTOR/Roi_Para_Clasificar/MAIZ_ROI_229_82_150321.shp
	/tmp/MAIZ_ROI_229_82_150321_train.shp
	/tmp/MAIZ_ROI_229_82_150321_test.shp
Created from data/bccba/VECTOR/Roi_Para_Clasificar/MANI_ROI_229_82_150201.shp
	/tmp/MANI_ROI_229_82_150201_train.shp
	/tmp/MANI_ROI_229_82_150201_test.shp
Created from data/bccba/VECTOR/Roi_Para_Clasificar/MANI_ROI_229_82_150